In [1]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [2]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/third-experiments/SDA_experiments/mlm', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/third-experiments/SDA_experiments/modules']


2024-06-25 05:53:59.506858: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-25 05:53:59.691689: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-25 05:54:00.476871: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import os
import tarfile

# Extract the dataset
tar_path = '../../datasets/domain_sentiment_data.tar.gz'
extract_path = '../../datasets/sorted_data_acl'

# with tarfile.open(tar_path, 'r:gz') as tar:
#     tar.extractall(path=extract_path)

# Load the reviews
def load_reviews(domain, label):
    reviews = []
    file_path = os.path.join(extract_path, domain, f'{label}.review')
    with open(file_path, 'r') as file:
        for line in file:
            reviews.append(line.strip())
    return reviews

# Example usage
books_positive_reviews = load_reviews('books', 'positive')
books_negative_reviews = load_reviews('books', 'negative')

print(f'Number of positive reviews in books: {len(books_positive_reviews)}')
print(f'Number of negative reviews in books: {len(books_negative_reviews)}')


Number of positive reviews in books: 37894
Number of negative reviews in books: 38043


In [8]:
import os
import tarfile
import random
import re
from datasets import Dataset, DatasetDict
from collections import Counter  # Ensure Counter is imported

# Function to load and parse reviews from XML
def load_reviews(domain, label):
    reviews = []
    file_path = os.path.join(extract_path, domain, f'{label}.review')
    with open(file_path, 'r', encoding='ISO-8859-1') as file:
        content = file.read()
        reviews_xml = content.split('</review>')[:-1]  # Split the file by review end tag
        for review_xml in reviews_xml:
            review_xml += '</review>'  # Add the closing tag back
            review_text_match = re.search(r'<review_text>(.*?)</review_text>', review_xml, re.DOTALL)
            if review_text_match:
                review_text = review_text_match.group(1).strip()
                reviews.append(review_text)
    return reviews

# Function to load unlabeled reviews
def load_unlabeled_reviews(domain):
    reviews = []
    file_path = os.path.join(extract_path, domain, 'unlabeled.review')
    with open(file_path, 'r', encoding='ISO-8859-1') as file:
        content = file.read()
        reviews_xml = content.split('</review>')[:-1]  # Split the file by review end tag
        for review_xml in reviews_xml:
            review_xml += '</review>'  # Add the closing tag back
            review_text_match = re.search(r'<review_text>(.*?)</review_text>', review_xml, re.DOTALL)
            if review_text_match:
                review_text = review_text_match.group(1).strip()
                reviews.append(review_text)
    return reviews

# Merge, label, and shuffle reviews
def get_labeled_reviews(domain):
    positive_reviews = load_reviews(domain, 'positive')
    negative_reviews = load_reviews(domain, 'negative')
    reviews = [{'review': review, 'label': 1} for review in positive_reviews] + \
              [{'review': review, 'label': 0} for review in negative_reviews]
    random.shuffle(reviews)
    return reviews

def get_unlabeled_reviews(domain):
    unlabeled_reviews = load_unlabeled_reviews(domain)
    reviews = [{'review': review} for review in unlabeled_reviews]
    return reviews

# Create DatasetDict object
domains = ['books', 'dvd', 'electronics', 'kitchen_&_housewares']
data_dict = {
    domain: {
        'labeled': get_labeled_reviews(domain),
        'unlabeled': get_unlabeled_reviews(domain)
    }
    for domain in domains
}

# Convert to datasets.Dataset
dataset_dict = {
    domain: Dataset.from_dict({
        'review': [entry['review'] for entry in data['labeled']],
        'label': [entry['label'] for entry in data['labeled']]
    })
    for domain, data in data_dict.items()
}
unlabeled_dataset_dict = {
    f"{domain}_unlabeled": Dataset.from_dict({
        'review': [entry['review'] for entry in data['unlabeled']]
    })
    for domain, data in data_dict.items()
}

# Combine labeled and unlabeled datasets
all_datasets = DatasetDict({**dataset_dict, **unlabeled_dataset_dict})

# Save datasets to disk
all_datasets.save_to_disk(f"{config.Config.DATASETS_SAVE_PATH}/SDA")

# To load the datasets back from disk
loaded_datasets = DatasetDict.load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/SDA")

# Verify the loaded datasets
for domain, dataset in loaded_datasets.items():
    print(f"Domain: {domain}")
    if "unlabeled" not in domain:
        balance = Counter(dataset['label'])
        print(f"  Class Balance: {balance}")
    print(f"  Total Samples: {len(dataset)}")
    print("  Sample Data:")
    print(dataset.shuffle(seed=42).select(range(1)).to_pandas()[['review', 'label'] if 'unlabeled' not in domain else ['review']])
    print("\n")


Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/34741 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13153 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/16785 [00:00<?, ? examples/s]

Domain: books
  Class Balance: Counter({0: 1000, 1: 1000})
  Total Samples: 2000
  Sample Data:
                                              review  label
0  This book started out good but went downhill q...      0


Domain: dvd
  Class Balance: Counter({0: 1000, 1: 1000})
  Total Samples: 2000
  Sample Data:
                                              review  label
0  I picked this set up despite reservations (res...      0


Domain: electronics
  Class Balance: Counter({0: 1000, 1: 1000})
  Total Samples: 2000
  Sample Data:
                                              review  label
0  Great product. It's two in one. You have it in...      1


Domain: kitchen_&_housewares
  Class Balance: Counter({1: 1000, 0: 1000})
  Total Samples: 2000
  Sample Data:
                                              review  label
0  For ideal coffee using a French press is the b...      1


Domain: books_unlabeled
  Total Samples: 6000
  Sample Data:
                                              re

In [9]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.colheader_justify', 'left')  # Align headers to the left
books_unlabeled_sample = loaded_datasets['dvd_unlabeled'].shuffle(seed=42).select(range(1)).to_pandas()
books_unlabeled_sample['review'] = books_unlabeled_sample['review'].str.strip()
# Display the sample
print(books_unlabeled_sample)

  review                                                                                                                                                                                                                                                                
0  Excellent movie. The plot was unique as only S. King can make it, the acting was excellent thanks to the great casting and the photography, scenery and costumes excellent. In a word - 'gripping' from bigining to end.\n\nThe DVD itself was also excellent quality


In [29]:
datasets.save_to_disk(f"{config.Config.DATASETS_SAVE_PATH}/SDA")

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

In [3]:
from datasets import load_from_disk,concatenate_datasets


adapter_name="SDA_mlm_inv_G"
dataset = load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/SDA")
dataset


DatasetDict({
    books: Dataset({
        features: ['review', 'label'],
        num_rows: 2000
    })
    dvd: Dataset({
        features: ['review', 'label'],
        num_rows: 2000
    })
    electronics: Dataset({
        features: ['review', 'label'],
        num_rows: 2000
    })
    kitchen_&_housewares: Dataset({
        features: ['review', 'label'],
        num_rows: 2000
    })
    books_unlabeled: Dataset({
        features: ['review'],
        num_rows: 6000
    })
    dvd_unlabeled: Dataset({
        features: ['review'],
        num_rows: 34741
    })
    electronics_unlabeled: Dataset({
        features: ['review'],
        num_rows: 13153
    })
    kitchen_&_housewares_unlabeled: Dataset({
        features: ['review'],
        num_rows: 16785
    })
})

In [4]:
dataset['books'][0:5]

{'review': ["I've read a lot of the books in the series, and Hillerman is one of my favorite mystery writers, but I have to say this one fell way short of his other mysteries.  My main complaint is the presence of several plot gaps and inconsistencies not characteristic of the author--his well-crafted mysteries almost always hold water, but not this one.  I whole-heartedly recommend most of Hillerman's other Jim Chee/Joe Leaphorn novels, but if you want a good entrance into his captivating style and skill, this is not the book to pick up.",
  'When Peter S. Lynch speaks, wise investors will listen.  This book covers the famous fund manager\'s career at the helm of Fidelity Magellan from 1977 to \'90, and post career into \'92.  It\'s far more introspective than "One Up On Wall Street" and it was no doubt meant to be for this purpose.  For example, there isn\'t nearly as much fundamental principles for stock picking outlined in this book as the former.  My belief is that the reader woul

In [6]:
import re

def clean_text(text):
    text = text.lower()  # Lowercase the text
    text = text.replace('\n', ' ').replace('\\n', ' ')  # Replace newline characters with spaces
    text = re.sub(r'[/]', ' ', text)  # Replace slashes with spaces
    text = re.sub(r'[^a-zA-Z0-9.,!?;:\s]', '', text)  # Keep letters, numbers, and common punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text


In [7]:
from datasets import DatasetDict, Dataset

def preprocess_function(examples):
    examples['review'] = [clean_text(review) for review in examples['review']]
    return examples

cleaned_datasets = dataset.map(preprocess_function, batched=True)


In [8]:
dataset['books'][0:5]

{'review': ["I've read a lot of the books in the series, and Hillerman is one of my favorite mystery writers, but I have to say this one fell way short of his other mysteries.  My main complaint is the presence of several plot gaps and inconsistencies not characteristic of the author--his well-crafted mysteries almost always hold water, but not this one.  I whole-heartedly recommend most of Hillerman's other Jim Chee/Joe Leaphorn novels, but if you want a good entrance into his captivating style and skill, this is not the book to pick up.",
  'When Peter S. Lynch speaks, wise investors will listen.  This book covers the famous fund manager\'s career at the helm of Fidelity Magellan from 1977 to \'90, and post career into \'92.  It\'s far more introspective than "One Up On Wall Street" and it was no doubt meant to be for this purpose.  For example, there isn\'t nearly as much fundamental principles for stock picking outlined in this book as the former.  My belief is that the reader woul

In [9]:
cleaned_datasets['books'][0:5]

{'review': ['ive read a lot of the books in the series, and hillerman is one of my favorite mystery writers, but i have to say this one fell way short of his other mysteries. my main complaint is the presence of several plot gaps and inconsistencies not characteristic of the authorhis wellcrafted mysteries almost always hold water, but not this one. i wholeheartedly recommend most of hillermans other jim chee joe leaphorn novels, but if you want a good entrance into his captivating style and skill, this is not the book to pick up.',
  'when peter s. lynch speaks, wise investors will listen. this book covers the famous fund managers career at the helm of fidelity magellan from 1977 to 90, and post career into 92. its far more introspective than one up on wall street and it was no doubt meant to be for this purpose. for example, there isnt nearly as much fundamental principles for stock picking outlined in this book as the former. my belief is that the reader would do best by reading one

In [11]:
cleaned_datasets.save_to_disk(f"{config.Config.DATASETS_SAVE_PATH}/SDA_cleaned")

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/34741 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13153 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/16785 [00:00<?, ? examples/s]

In [69]:
reload(config)

    
filtered_target = cleaned_datasets['electronics'].shuffle(seed=42)
unlabled_filtered_target = cleaned_datasets['electronics_unlabeled'].shuffle(seed=42)
#filtered_target = shuffled_filtered_target.train_test_split(test_size=0.1)

unlabled_filtered_target[4]
 

{'review': 'i was looking for a mouse to use with an hp livestrong laptop. i liked the size of the mouse, as well as the price. positive reviews made me decide to give it a try. ive had the mouse for two months now, and so far it has been working well. the only complaint that i have: it tends to lose bluetooth connection randomly. sometime i have to search for it using search for devices in range function on my laptop. on the positive side, i like its size and feel i wouldnt want to use it all the time but for a laptop its great. it comes with a usb charger and a storage bag. great part is that is doesnt need batteries and you can use it while its charging. you would need bluetooth to use it make sure you have it; otherwise, buy a wireless mouse. it works pretty precisely much better than a wireless dell mouse; it doesnt skip at all. installation was easy, but took me a few minutes to figure it out because it was my first bluetooth device that i was connecting to the laptop. overall, i

In [70]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoModelForMaskedLM.from_pretrained(
    config.Config.MODEL_NAME,
)
init(model)
reload(fn)
fn.print_trainable_parameters(model)

/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFor

trainable params: 109514298 || all params: 109514298 || trainable%: 100.0


In [71]:
from adapters import ConfigUnion, LoRAConfig, PrefixTuningConfig, SeqBnConfig,SeqBnInvConfig,AdapterConfig,LoRAConfig


adapter_config = SeqBnInvConfig(reduction_factor=2)

model.add_adapter(adapter_name, config=adapter_config)

model.train_adapter([adapter_name])
model.active_adapters = adapter_name
print(model.adapter_summary())
fn.print_trainable_parameters(model)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
SDA_mlm_inv_G            bottleneck        7,387,776       6.785       1       1
--------------------------------------------------------------------------------
Full model                               108,891,648     100.000               0
trainable params: 7979904 || all params: 116902074 || trainable%: 6.82614407679371


In [72]:


def count_words(dataset):
    total_premise_words = 0
    total_hypothesis_words = 0

    # Iterate through each record in the dataset
    for entry in dataset:
        # Split the 'premise' and 'hypothesis' fields on spaces to count words
        premise_words = entry['review'].split()

        # Sum up the word counts
        total_premise_words += len(premise_words)

    return total_premise_words

In [73]:
count_words(unlabled_filtered_target)

1439083

In [74]:
from transformers import AutoTokenizer

reload(processed)
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

def tokenize_dataset(data,tokenizer):
    def tokenize_function(examples):
        result = tokenizer(examples['review']) # no trunccation or padding cuz it is mlm
        if tokenizer.is_fast:
            result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
        return result


    # Use batched=True to activate fast multithreading!
    tokenized_datasets = data.map(
        tokenize_function, batched=True, 
    )
    tokenized_datasets = tokenized_datasets.remove_columns(['review'])

    return tokenized_datasets

tokenized_dataset= tokenize_dataset(unlabled_filtered_target,tokenizer)
tokenized_dataset

Map:   0%|          | 0/13153 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (595 > 512). Running this sequence through the model will result in indexing errors


Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
    num_rows: 13153
})

In [76]:
chunk_size = 512
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset[444:470]

results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'
'>>> Chunk length: 512'


In [77]:
# we group texts and chunk them
lm_datasets = tokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
lm_datasets

Map:   0%|          | 0/13153 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 3411
})

In [78]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets[i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] flakey [MASK] like nearly all of dlinks [MASK] products. some days [MASK] works, connecting [MASK] to the dlink wpapsk [MASK]r i have, indy sometimes it wont connect at all. the 100mb [MASK] wired ethernet always works, [MASK], and works very well [MASK] a nice wired camera [MASK] that [MASK] will do wireless as well [SEP] [CLS] disappointing [MASK]. frequent reinstall necessary because it loses its linking id patriots more trouble [MASK] it is worth. i went back to cords [SEP] [CLS] i [MASK] 2 of the cdmavica [MASK] [MASK] the cd [MASK]0 uses these disks [MASK] with no problems. however with the cd500, about 2 3 [MASK] the disks have problems [MASK] i know it is not my camera causing this problem, i know several people with the same camera that have the same problem. if [MASK] have a cd500, [MASK] the [MASK] [MASK] [MASK] [MASK], i [MASK] not had a problem with any of them. [SEP] [CLS] crosses 1 gb card took all [MASK] pictures on a trip to hawaii, over [MASK] [SEP] [CLS] 

In [79]:

downsampled_dataset = lm_datasets.train_test_split(
    test_size=0.1, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 3069
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 342
    })
})

In [80]:

reload(fn)
trainer = fn.train_mlm_model(model,adapter_name,data_collator,tokenizer, downsampled_dataset['train'],downsampled_dataset['test'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [81]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/11 [00:00<?, ?it/s]

>>> Perplexity: 23.34


In [82]:
trainer.train()

  0%|          | 0/1920 [00:00<?, ?it/s]

{'loss': 3.1794, 'grad_norm': 1.1464124917984009, 'learning_rate': 1.9e-05, 'epoch': 0.99}


  0%|          | 0/11 [00:00<?, ?it/s]

{'eval_loss': 2.8596105575561523, 'eval_runtime': 2.3706, 'eval_samples_per_second': 144.266, 'eval_steps_per_second': 4.64, 'epoch': 1.0}
{'loss': 2.9097, 'grad_norm': 1.1410419940948486, 'learning_rate': 3.8e-05, 'epoch': 1.98}


  0%|          | 0/11 [00:00<?, ?it/s]

...

  0%|          | 0/11 [00:00<?, ?it/s]

{'eval_loss': 2.2944138050079346, 'eval_runtime': 4.5377, 'eval_samples_per_second': 75.368, 'eval_steps_per_second': 2.424, 'epoch': 20.0}


Overwriting existing adapter 'SDA_mlm_inv_G'.


{'train_runtime': 1783.2134, 'train_samples_per_second': 34.421, 'train_steps_per_second': 1.077, 'train_loss': 2.5333909412225086, 'epoch': 20.0}


TrainOutput(global_step=1920, training_loss=2.5333909412225086, metrics={'train_runtime': 1783.2134, 'train_samples_per_second': 34.421, 'train_steps_per_second': 1.077, 'train_loss': 2.5333909412225086, 'epoch': 20.0})

In [83]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/11 [00:00<?, ?it/s]

>>> Perplexity: 9.76


In [84]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)


: 

#### This is a report of mlm for goverment domain using whole genre as target and then splitt after tokenization with seed 42. I used computer metrix for preplexity and early stopping 
The results before:
>>> Perplexity: 16.32

The results after:
>>> Perplexity: 4.99